# _Conv1d, Conv2d_

In [9]:
import torch
from torch import nn
from torchsummaryX import summary
from complexNN.complexFunction import ComplexMaxPool2d, ComplexDropout2d
from complexNN.complexActivation import cRelu
from complexNN.complexLayer import complexConv2d, complexLinear


class CNN(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv1 = complexConv2d(in_channels, out_channels, padding='same')
        self.conv2 = complexConv2d(out_channels, out_channels, padding='same')
        self.dropout = ComplexDropout2d()
        self.maxpool = ComplexMaxPool2d(kernel_size=3, stride=1)
        self.fc = complexLinear(252*252, 10)
        self.ac = cRelu()

    def forward(self, x):
        x = self.dropout(self.ac(self.conv1(x)))
        x = self.maxpool(x)
        x = self.dropout(self.ac(self.conv2(x)))
        x = self.maxpool(x)
        x = x.reshape(x.shape[0], x.shape[1], -1)
        return self.fc(x)


if __name__ == '__main__':
    batch_size, in_channels, out_channels, seq_len = 1, 3, 16, 10
    H, W = 256, 256
    conv2d_tensor = torch.rand((batch_size, in_channels, H, W))
    cnn = CNN(in_channels, out_channels)
    print(cnn(conv2d_tensor).shape)
    summary(cnn, conv2d_tensor)

torch.Size([1, 16, 10])
             Kernel Shape       Output Shape   Params    Mult-Adds
Layer                                                             
0_conv1     [3, 16, 3, 3]  [1, 16, 256, 256]    432.0   28.311552M
1_ac                    -  [1, 16, 256, 256]        -            -
2_dropout               -  [1, 16, 256, 256]        -            -
3_maxpool               -  [1, 16, 254, 254]        -            -
4_conv2    [16, 16, 3, 3]  [1, 16, 254, 254]   2.304k  148.644864M
5_ac                    -  [1, 16, 254, 254]        -            -
6_dropout               -  [1, 16, 254, 254]        -            -
7_maxpool               -  [1, 16, 252, 252]        -            -
8_fc          [63504, 10]        [1, 16, 10]  635.05k      635.04k
-------------------------------------------------------------------
                           Totals
Total params             637.786k
Trainable params         637.786k
Non-trainable params          0.0
Mult-Adds             177.591456M


C:\Users\Liao\anaconda3\lib\site-packages\torchsummaryX\torchsummaryX.py:101: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_sum = df.sum()


# _MLP_

In [10]:
import torch
from complexNN.complexLayer import complexMLP


if __name__ == '__main__':
    batch_size, input_size, hidden_size, output_size = 10, 10, 20, 15
    input_tensor = torch.rand((batch_size, input_size), dtype=torch.cfloat)
    mlp = complexMLP(input_size, hidden_size, output_size, num_layers=3)
    out = mlp(input_tensor)
    print(out.shape)
    summary(mlp, input_tensor)

torch.Size([10, 15])
                                Kernel Shape Output Shape Params Mult-Adds
Layer                                                                     
0_input_layer                       [10, 20]     [10, 20]  220.0     200.0
1_activation                               -     [10, 20]      -         -
2_hidden_layers.complexLinear_0     [20, 20]     [10, 20]  420.0     400.0
3_bn.BatchNorm1d_real_bn                [20]     [10, 20]   40.0      20.0
4_bn.BatchNorm1d_imag_bn                [20]     [10, 20]   40.0      20.0
5_activation                               -     [10, 20]      -         -
6_hidden_layers.complexLinear_1     [20, 20]     [10, 20]  420.0     400.0
7_bn.BatchNorm1d_real_bn                [20]     [10, 20]      -      20.0
8_bn.BatchNorm1d_imag_bn                [20]     [10, 20]      -      20.0
9_activation                               -     [10, 20]      -         -
10_output_layer                     [20, 15]     [10, 15]  315.0     300.0
----

C:\Users\Liao\anaconda3\lib\site-packages\torchsummaryX\torchsummaryX.py:101: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_sum = df.sum()


# _RNN, GRU, LSTM_

In [14]:
import torch
from complexNN.complexRNN import complexRNN, complexGRU, complexLSTM


if __name__ == '__main__':
    batch_size, input_size, hidden_size, seq_len, num_layers = 10, 10, 20, 1, 3
    input_tensor = torch.rand((seq_len, batch_size, input_size), dtype=torch.cfloat)
    h0, c0 = torch.zeros((num_layers, batch_size, hidden_size)), torch.zeros((num_layers, batch_size, hidden_size))

    rnn = complexRNN(input_size, hidden_size, num_layers)
    gru = complexGRU(input_size, hidden_size, num_layers)
    lstm = complexLSTM(input_size, hidden_size, num_layers)

    rnn_out, _ = rnn(input_tensor, h0)
    gru_out, _ = gru(input_tensor, h0)
    lstm_out, _ = lstm(input_tensor, (h0, c0))

    print(rnn_out.shape, gru_out.shape, lstm_out.shape)
    summary(rnn, input_tensor, h0)
    summary(gru, input_tensor, h0)
    summary(lstm, input_tensor, (h0, c0))

torch.Size([1, 10, 20]) torch.Size([1, 10, 20]) torch.Size([1, 10, 20])
                               Kernel Shape Output Shape  Params  Mult-Adds
Layer                                                                      
0_rnn_cells.0.complexLinear_Wx     [10, 20]     [10, 20]     220        200
1_rnn_cells.0.complexLinear_Wh     [20, 20]     [10, 20]     420        400
2_rnn_cells.1.complexLinear_Wx     [20, 20]     [10, 20]     420        400
3_rnn_cells.1.complexLinear_Wh     [20, 20]     [10, 20]     420        400
4_rnn_cells.2.complexLinear_Wx     [20, 20]     [10, 20]     420        400
5_rnn_cells.2.complexLinear_Wh     [20, 20]     [10, 20]     420        400
---------------------------------------------------------------------------
                      Totals
Total params            2320
Trainable params        2320
Non-trainable params       0
Mult-Adds               2200
                                Kernel Shape Output Shape  Params  Mult-Adds
Layer                 

C:\Users\Liao\anaconda3\lib\site-packages\torchsummaryX\torchsummaryX.py:101: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_sum = df.sum()
C:\Users\Liao\anaconda3\lib\site-packages\torchsummaryX\torchsummaryX.py:101: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_sum = df.sum()
C:\Users\Liao\anaconda3\lib\site-packages\torchsummaryX\torchsummaryX.py:101: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_sum = df.sum()
